In [174]:
import pandas as pd


# Análise e Limpeza do DataSet de Teste

In [175]:
test = pd.read_csv('test.csv')

In [176]:
test.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


In [177]:
test.describe()

,enrollee_id,city_development_index,training_hours
count,2129.000000,2129.000000,2129.000000
mean,16861.614843,0.824984,64.983091
std,9576.846029,0.125074,60.238660
min,3.000000,0.448000,1.000000
25%,8562.000000,0.698000,23.000000
50%,16816.000000,0.903000,47.000000
75%,25129.000000,0.920000,86.000000
max,33353.000000,0.949000,334.000000


In [178]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             2129 non-null   int64  
 1   city                    2129 non-null   object 
 2   city_development_index  2129 non-null   float64
 3   gender                  1621 non-null   object 
 4   relevent_experience     2129 non-null   object 
 5   enrolled_university     2098 non-null   object 
 6   education_level         2077 non-null   object 
 7   major_discipline        1817 non-null   object 
 8   experience              2124 non-null   object 
 9   company_size            1507 non-null   object 
 10  company_type            1495 non-null   object 
 11  last_new_job            2089 non-null   object 
 12  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 216.4+ KB


In [179]:
test.set_index('enrollee_id', inplace = True)

In [180]:
test.gender.unique()

array(['Male', 'Female', nan, 'Other'], dtype=object)

In [181]:
test.gender.isna().sum()

508

In [182]:
test[test.gender.isna()]

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
enrollee_id,,,,,,,,,,,,
21465,city_21,0.624,NaN,Has relevent experience,no_enrollment,Graduate,STEM,<1,100-500,Pvt Ltd,1,11
24372,city_98,0.949,NaN,Has relevent experience,no_enrollment,Masters,STEM,4,100-500,Pvt Ltd,1,134
14070,city_103,0.920,NaN,No relevent experience,no_enrollment,Graduate,STEM,5,NaN,NaN,never,10
24914,city_21,0.624,NaN,Has relevent experience,Full time course,Graduate,STEM,13,1000-4999,Pvt Ltd,1,125
21514,city_21,0.624,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,NaN,Pvt Ltd,4,23
...,...,...,...,...,...,...,...,...,...,...,...,...
19499,city_75,0.939,NaN,No relevent experience,NaN,High School,NaN,2,NaN,NaN,never,44
22744,city_24,0.698,NaN,Has relevent experience,no_enrollment,Masters,STEM,15,50-99,Pvt Ltd,>4,157
6294,city_73,0.754,NaN,Has relevent experience,no_enrollment,High School,NaN,>20,50-99,Pvt Ltd,2,31


In [183]:
test.dropna(subset = ['gender'], axis = 0, inplace = True)

In [184]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621 entries, 32403 to 12215
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    1621 non-null   object 
 1   city_development_index  1621 non-null   float64
 2   gender                  1621 non-null   object 
 3   relevent_experience     1621 non-null   object 
 4   enrolled_university     1603 non-null   object 
 5   education_level         1596 non-null   object 
 6   major_discipline        1404 non-null   object 
 7   experience              1619 non-null   object 
 8   company_size            1194 non-null   object 
 9   company_type            1181 non-null   object 
 10  last_new_job            1615 non-null   object 
 11  training_hours          1621 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 164.6+ KB


In [185]:
test.company_size.unique()

array(['<10', nan, '10/49', '10000+', '50-99', '100-500', '1000-4999',
       '500-999', '5000-9999'], dtype=object)

In [186]:
test.company_size.isna().sum()

427

In [187]:
test.company_size.dtype

dtype('O')

In [188]:
test.company_size.value_counts().index[0] #a moda da coluna

'50-99'

In [189]:
test.company_size.fillna(test.company_size.value_counts().index[0], inplace = True)

In [190]:
test.company_size.unique()

array(['<10', '50-99', '10/49', '10000+', '100-500', '1000-4999',
       '500-999', '5000-9999'], dtype=object)

In [191]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621 entries, 32403 to 12215
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    1621 non-null   object 
 1   city_development_index  1621 non-null   float64
 2   gender                  1621 non-null   object 
 3   relevent_experience     1621 non-null   object 
 4   enrolled_university     1603 non-null   object 
 5   education_level         1596 non-null   object 
 6   major_discipline        1404 non-null   object 
 7   experience              1619 non-null   object 
 8   company_size            1621 non-null   object 
 9   company_type            1181 non-null   object 
 10  last_new_job            1615 non-null   object 
 11  training_hours          1621 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 164.6+ KB


In [192]:
test.company_type.unique()

array([nan, 'Pvt Ltd', 'Funded Startup', 'Other', 'Public Sector',
       'Early Stage Startup', 'NGO'], dtype=object)

In [193]:
test.groupby(['company_type'])['company_size'].agg(pd.Series.mode)

company_type
Early Stage Startup        <10
Funded Startup           50-99
NGO                    100-500
Other                    50-99
Public Sector            50-99
Pvt Ltd                  50-99
Name: company_size, dtype: object

In [194]:
test.groupby(['company_size'])['company_type'].agg(pd.Series.mode)

company_size
10/49        Pvt Ltd
100-500      Pvt Ltd
1000-4999    Pvt Ltd
10000+       Pvt Ltd
50-99        Pvt Ltd
500-999      Pvt Ltd
5000-9999    Pvt Ltd
<10          Pvt Ltd
Name: company_type, dtype: object

In [195]:
test.company_type.isna().sum()

440

In [196]:
test[test['company_size'] == '<10'].company_type.isna().sum()

9

In [197]:
test[test['company_size'] == '50-99'].company_type.isna().sum()

397

In [198]:
test[test['company_size'] == '100-500'].company_type.isna().sum()

12

In [199]:
test[test['company_size'] == '10/49'].company_type.isna().sum()

8

In [200]:
test[test['company_size'] == '1000-4999'].company_type.isna().sum()

6

In [201]:
test[test['company_size'] == '10000+'].company_type.isna().sum()

3

In [202]:
test[test['company_size'] == '500-999'].company_type.isna().sum()

4

In [203]:
test[test['company_size'] == '5000-9999'].company_type.isna().sum()

1

In [204]:
test.last_new_job.value_counts().index[0] #a moda da coluna

'1'

In [205]:
test.last_new_job.fillna(test.last_new_job.value_counts().index[0], inplace = True)

In [206]:
test.enrolled_university.isna().sum()

18

In [207]:
test.enrolled_university.value_counts().index[0] #moda

'no_enrollment'

In [208]:
test.enrolled_university.fillna(test.enrolled_university.value_counts().index[0], inplace = True)

In [209]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621 entries, 32403 to 12215
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    1621 non-null   object 
 1   city_development_index  1621 non-null   float64
 2   gender                  1621 non-null   object 
 3   relevent_experience     1621 non-null   object 
 4   enrolled_university     1621 non-null   object 
 5   education_level         1596 non-null   object 
 6   major_discipline        1404 non-null   object 
 7   experience              1619 non-null   object 
 8   company_size            1621 non-null   object 
 9   company_type            1181 non-null   object 
 10  last_new_job            1621 non-null   object 
 11  training_hours          1621 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 164.6+ KB


In [210]:
test.education_level.isna().sum()

25

In [211]:
test.education_level.value_counts().index[0] #moda

'Graduate'

In [212]:
test.education_level.fillna(test.education_level.value_counts().index[0], inplace = True)

In [213]:
test.major_discipline.isna().sum()

217

In [215]:
test.major_discipline.value_counts().index[0] #moda

'STEM'

In [216]:
test.major_discipline.fillna(test.major_discipline.value_counts().index[0], inplace = True)

In [219]:
test.experience.isna().sum()

2

In [220]:
test.experience.value_counts().index[0] #moda

'>20'

In [221]:
test.experience.fillna(test.experience.value_counts().index[0], inplace = True)

In [222]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621 entries, 32403 to 12215
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    1621 non-null   object 
 1   city_development_index  1621 non-null   float64
 2   gender                  1621 non-null   object 
 3   relevent_experience     1621 non-null   object 
 4   enrolled_university     1621 non-null   object 
 5   education_level         1621 non-null   object 
 6   major_discipline        1621 non-null   object 
 7   experience              1621 non-null   object 
 8   company_size            1621 non-null   object 
 9   company_type            1181 non-null   object 
 10  last_new_job            1621 non-null   object 
 11  training_hours          1621 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 164.6+ KB
